(raga-recognition)=
# Automatic raga recognition

Automatic raga recognition works intend to identify the raga that is performed in an Indian Art Music recording. Historically, raga recognition approaches have been based on melodic features from the vocal line, trying to identify relationships between the sung patterns and scales. More recently, this problem is being addressed by passing the melodic features through DL models that learn the relationships within the data and learn to discriminate between the different ragas in a dataset {cite}`ross_raga_2017, madhusudhan_raga_2019, john_raga_2020, clayton_raga_2022`.

This is a core task within the melodic analysis of Indian Art Music, and most of the research on this topic build on top of the Dunya Carnatic and Hindustani corpora {cite}`iam_corpora`.

In [ ]:
%pip install compiam

## Importing compiam to the project
import compiam

# Import extras and supress warnings to keep the tutorial clean
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

Let's first list the available tools for the task of raga recognition.

In [ ]:
compiam.melody.raga_recognition.list_tools()

We already see that the available tool appears with ``*`` at the end of the name, which indicates that it may be loaded using the ``compiam.load_models()`` wrapper. Observing [the entry of this tool in the `compiam` documentation](https://mtg.github.io/compIAM/source/melody.html#module-compiam.melody.raga_recognition.deepsrgm), we note that an optional dependency is needed to properly load this tool. Let's install it:

In [ ]:
%pip install torch==1.8.0

DEEPSRGM {cite}`madhusudhan_raga_2019` is a DL-based model that uses melodic features to automatically identify the raga from an Indian Art Music recording. In the original paper, the authors use the Indian Art Music Raga Dataset {cite}`raga_dataset`, and report accuracy higher than 90%.

In [ ]:
# Importing a tool for automatic recognition
from compiam.melody.raga_recognition import DEEPSRGM

# We initialize an instance of the tool
deepsrgm = DEEPSRGM()

However, this is a non-trained instance of DEEPSRGM, and as suggested by the output of ``compiam.melody.raga_recognition.list_tools()``, said model appears with a ``*`` in the name so a trained instance may be loaded using the wrapper for models.

```{tip}
You can also browse the available pre-trained models by running ``compiam.list_models()``. That will also help getting the 
```

In [ ]:
compiam.list_models()

In [ ]:
# We initialize an instance of the tool
deepsrgm = compiam.load_model("melody:deepsrgm")

# Showing raga mapping
deepsrgm.mapping

Therefore, the provided pre-trained model has been trained targetting to the ragas included in this mapping. Therefore, let's select some examples that are tagged with raga instances included in the default mapping to evaluate this tool and showcase how it works.

In [ ]:
# Initialize saraga Carnatic on our example audio folder
saraga_carnatic = compiam.load_dataset("saraga_carnatic", data_home="./../audio/")

```{note}
Please note that for space purposes, we are not loading the entire dataset but just a single recording. However, we can still use the dataset loader, which in this case will properly work as long as we only retrieve the data for this particular recording.
```

The recording we have included beforehand for tutoring purposes is Sri Raghuvara Sugunaalaya, performed by Sanjay Subrahmanyan, at Vani Mahal. We will load this track by first parsing its id in the data loader. Using the loader on top of the full dataset, we have obtained its dataloder id: ``109_Sri_Raghuvara_Sugunaalaya``

In [ ]:
track_data = saraga_carnatic.load_tracks()["109_Sri_Raghuvara_Sugunaalaya"]
pprint("This recordings includes raaga {}, which has unique Dunya ID {}"\
    .format(track_data.metadata["raaga"][0]["name"], track_data.metadata["raaga"][0]["uuid"]))

**Nice!** Raga Bhairavi is both in this recording and included in the mapping of DEEPSRGM, therefore considered during the training process.

In [ ]:
# Computing features for the given recording
feat = deepsrgm.get_features(track_data.audio_path)

# Predicting raga!
deepsrgm.predict(feat)